# VDS 1022/i Oscilloscope

To install from this folder :

`python -m pip install .`

Help:

In [ ]:
from vds1022 import *
help(vds1022)

In [ ]:
from vds1022 import *
help(vds1022.Frames)

In [ ]:
from vds1022 import *
help(vds1022.Frame)

Note that the first call of `VDS1022()` will take a few seconds since it has to load the FPGA firmware.
The device will remain connected unless `.dispose()` is called or if the kernel is restarted. Interrupting the kernel (`I`,`I`) will not terminate the connection.

### Examples :
1. [Read voltage](#1.-Read-voltage)
2. [Acquire a signal](#2.-Acquire-a-signal)
3. [Live plotting](#3.-Live-plotting)
4. [Data logging](#4.-Data-logging)
5. [Continuous sampling](#5.-Continuous-sampling)
6. [Decoding](#6.-Decoding)
7. [Spectrum analysis](#7.-Spectrum-analysis)
8. [Calibration](#8.-Calibration)
9. [Release device](#9.-Release-device)

## 1. Read voltage

Read RMS voltage on channel 1  
Metrics: `rms, avg, max, min, std, freq, median, percentile`

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='10v', offset=1/10, probe='x10')
frames = dev.fetch()
print("%s Vrms" % frames.ch1.rms())

Plotting

In [ ]:
frames.plot()

Read in a loop (interrupt to stop)

In [ ]:
try:
    for frames in dev.fetch_iter(freq=2, autorange=False):
        print('CH1:%sv     ' % frames.ch1.rms(), end='\r')
except KeyboardInterrupt: pass

## 2. Acquire a signal

Autoset the device and plot the signals

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
frames = dev.autoset().fetch()
frames.plot()

Generate descriptive statistics.

In [ ]:
frames.describe()

Convert to [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe)

In [ ]:
frames.to_dataframe()

Fetch a triggered signal and plot.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_timerange('10ms')
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='2v', position=1/2)
frames = dev.fetch()
frames.plot()

## 3. Live plotting


Plot the signals with AUTO auto sweep mode

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('250k')
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='1v', position=1/2, sweep=AUTO)
dev.plot(freq=2);

Plot the signals with NORMAL sweep mode

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('250k')
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='1v', position=1/2, sweep=NORMAL)
dev.plot(freq=2);

Stop acquisition

In [ ]:
dev.stop();

## 4. Data logging
This section provides examples to measure a signal at a defined interval.

Stream the RMS voltage to a plot every second.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='10v', offset=1/10, probe='x10')
dev.stream(freq=2).rms().plot();

Stream the average voltage to a streamz dataframe every second.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='20v', offset=1/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.stream(freq=1).avg().to_dataframe()

Stream the average voltage to stdout every second.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='20v', offset=1/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.stream(freq=1).avg().sink(print);

Device roll mode

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_timerange('60s', roll=True)
dev.set_channel(CH1, range='10v', offset=1/10, probe='x10')
dev.plot(freq=1);

In [ ]:
dev.stop();

Help:

In [ ]:
from vds1022 import *
help(vds1022.Stream)

## 5. Continuous sampling

This section provides examples to aquire and plot continuous samples without interruption for a defined duration.  
While this device can acquire sampling frames at 100Ms/s, the maximum continuous sampling rate is around 100Ks/s.
If the amount of collected data is too consequent, the ploting will have to go through rasterization.

Acquire continuously 10 samples per period for a 1Khz signal and plot :

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('10k')  # 1K samples per seconds * 10 samples
dev.set_channel(CH1, range='10v', offset=2/10, probe='x10')
dev.set_trigger(CH1, EDGE, FALL, level='2.5v')

frames = dev.read('1s')
frames.plot()
print("Samples: %s" % frames.ch1.size)

In [ ]:
frames.slice('0.5s', '0.6s').plot()

In [ ]:
frames.to_dataframe()

Acquire continuously 10 samples per period for a 1Khz signal and plot with rasterisation :

In [ ]:
from vds1022 import *
import hvplot.pandas

dev = VDS1022(debug=0)
dev.set_sampling('10k')  # 1K samples per seconds * 10 samples
dev.set_channel(CH1, range='10v', offset=2/10, probe='x10')
dev.set_trigger(CH1, EDGE, FALL, level='2.5v')

df = dev.read('1s').to_dataframe()
df.hvplot(rasterize=True).opts(width=800, ylim=dev.ylim(), colorbar=False)

## 6. Decoding

TTL

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('10k')  # 1K samples per seconds * 10 samples
dev.set_channel(CH1, range='10v', offset=2/10, probe='x10')
dev.set_trigger(CH1, EDGE, FALL, level='2.5v')

frames = dev.read('1s')
frames.ch1.to_ttl()

I2C

In [ ]:
from vds1022 import *

dev = VDS1022()
dev.set_sampling('10k')  # signal frequency * n samples
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')  # sda
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')  # scl
dev.set_trigger(CH1, EDGE, FALL, position=1/20, level='1.2v')
frames = dev.fetch()
frames.plot()

for msg in frames.decode_i2c():
    print(msg, msg.data)

## 7. Spectrum analysis

Setup device for 1kHz signal

In [ ]:
from vds1022 import *

dev = VDS1022()
dev.set_sampling('100k')  #  1kHz * 100 samples
# dev.set_sampling('100k')  #  1kHz * 100 samples
dev.set_channel(CH1, range='10v', probe='x10', coupling=AC)
dev.set_trigger(CH1, level=0)  # align the phase angle
dev.wait(1)  # wait for the device AC coupling capacitor to charge

Plot spectrum

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# acquire samples and compute the discrete Fourier Transform
frame = dev.fetch().ch1
xf, yf, pf = frame.fft(window=np.blackman)  # window: None, np.hanning, np.blackman ...

# plot
plt.rcParams['axes.grid'] = True
fig, axs = plt.subplots(4, 1, figsize=(12, 4 * 2.3))
axs[0].set_xlabel('Time [ms]')
axs[1].set_xlabel('Frequency [kHz]')
axs[0].set_ylabel('Input [V]')
axs[1].set_ylabel('Magnitude [Vrms]')
axs[2].set_ylabel('Magnitude [dBV]')
axs[3].set_ylabel('Phase [rad]')
axs[2].yaxis.set_ticks(np.arange(-100, 20, 20));
axs[3].yaxis.set_ticks(np.arange(-2*np.pi, 2*np.pi, np.pi/2));
axs[0].plot(frame.x() * 1000, frame.y())
axs[1].plot(xf/1000, yf)
axs[2].plot(xf/1000, 20 * np.log10(yf))
axs[3].plot(xf/1000, pf)
plt.tight_layout()

Extract components from signal.


In [ ]:
for f, v, p in frame.frequencies(threshold='5%', window=None):
    print("%6s Hz  %5.2f Vrms  %5.2f rad" % (f, v, p))

## 8. Calibration
Wait a few minutes for the temperature of the device to stabilize before running the calibration.  
Once done, the calibration is saved to a file which will be used from now on.  
To restore the factory calibration, simply delete the file.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.calibrate()

## 9. Release device

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.dispose()